In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from numba import njit
import feed
import datetime
import matplotlib.pyplot as plt
import copy
import random
from tqdm.notebook import tqdm


#### 優化程式 
    1. 試著將pandas的東西都轉換成vectorbt
    2. 試著將for loop可以vectorize的都轉換

#### 關於vectorbt好用的函數整理
1. series.vbt.rolling_split 可以分割成train, valid, test的rolling
2. vbt.range_splt 可以分割成很多個range包 (分成幾包，每包多少資料，會自動將資料重疊)
3. vbt.LEXLB.run 可以找到local extrema用來給ML



In [2]:
def init():
    vbt.settings['plotting']['layout']['width'] = 800
    vbt.settings['plotting']['layout']['height'] = 400
    pd.options.plotting.backend = "matplotlib"
    plt.rcParams['font.family'] = ['Microsoft JhengHei']  # 中文標籤
    plt.rcParams['axes.unicode_minus'] = False  # 負號
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    pd.options.display.float_format = '{:,.4f}'.format


def add_factors(name, df, stk_len, stk_num, func):
    new_col = list(zip([name]*stk_len, stk_num))
    new_df = eval(func)
    new_df.columns = new_col
    new_df = new_df.replace([np.inf, -np.inf, np.nan],
                            0).mask(new_df > 5, 5).mask(new_df < -5, -5)
    df = pd.concat([df, new_df], axis=1)
    return df


def calculate_value(df, dic, data_list, reverse_list, stk_len, stk_num):
    # 行業名稱
    df_value = copy.deepcopy(df)
    for sname in dic['指數彙編'].iloc[:, 0].unique():
        # 由行業名稱選定股票代碼
        symbols = dic['指數彙編'][dic['指數彙編'].iloc[:, 0]
                              == sname].iloc[:, -2].astype(str).values
        # 從大表中提取單一行業中所有股票的單一Factor

        for factor in data_list:  # 這邊沒再對MA做比較(這樣額外Factors都不會做比較)
            table = df.loc[:, factor].loc[:, symbols].fillna(method='ffill')
            # 計算單一個股目前資料水準
            table1 = (table - table.rolling(500).mean()) / \
            table.rolling(500).std()
            # 計算行業中心目前資料水平
            table2 = table.subtract(table.mean(axis=1), axis=0).divide(
                table.std(axis=1), axis=0)
            # 處理錯誤值與極端值
            table1.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
            table2.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
            table1 = table1.mask(table1 > 5, 5).mask(table1 < -5, -5)
            table2 = table2.mask(table2 > 5, 5).mask(table2 < -5, -5)
            # 個別與行業相加
            ta = table1+table2
            ta.columns = list(zip([factor]*len(table.columns), table.columns))
            df_value.loc[:, ta.columns] = ta

    # 先將反向的乘上-1
    reverse = reverse_list
    for re in reverse:
        col = list(zip([re]*stk_len, stk_num))
        df_value.loc[:, col] = df_value.loc[:, col] * -1
    return df_value

    # 計算合計分數


def calculate_scores(df_value, stk_idx, stk_num, **params):
    scores = pd.DataFrame(index=stk_idx, columns=stk_num).fillna(0)
    for d in df_value.columns.get_level_values(0).unique():
        scores += df_value[d] * params[d]
    return scores


def backtesting(df, scores, bins, stk_idx, stk_num, ls: str='onlylong', pplot: bool=False, freq='D', init_cash: int=10000000, returnbs: bool=False):
    return_df = list()
    # long = dict()
    # short = dict()
    bins = bins
    labels = [str(i) for i in range(1, bins+1)]
    rt = df['還原收盤價'].shift(-1).groupby(pd.Grouper(freq=freq)).tail(
        1).pct_change().shift(-1).replace([np.inf, -np.inf, np.nan], 0)
    bs_l = pd.DataFrame(index=stk_idx, columns=stk_num).fillna(
        0).groupby(pd.Grouper(freq=freq)).tail(1)
    if ls != 'onlylong':
        bs_s = pd.DataFrame(index=stk_idx, columns=stk_num).fillna(
            0).groupby(pd.Grouper(freq=freq)).tail(1)

    for idx, row in scores.groupby(pd.Grouper(freq=freq)).tail(1).iterrows():
        try:
            bs_l.loc[idx, row[pd.cut(row, bins=bins, labels=labels)
                            == labels[-1]].index.values] = 1
            if ls != 'onlylong':
                bs_s.loc[idx, row[pd.cut(row, bins=bins, labels=labels)
                                == labels[0]].index.values] = -1
        # long[idx] = row[pd.cut(row, bins=bins, labels=labels)
        #                 == labels[-1]].index.values
        # short[idx] = row[pd.cut(row, bins=bins, labels=labels)
        #                 == labels[0]].index.values
        except:
            pass
        if pplot:
            se = rt.loc[idx].groupby(pd.cut(row, bins=bins, labels=labels)).mean()
            return_df.append(se)
    if pplot:
        return_df = pd.concat(return_df, axis=1).T
        return_df.cumsum().plot(figsize=(16, 9)).plot(title='Bins Return')
        plt.show()

    per_cash = init_cash / bs_l.sum(axis=1)
    # long
    bs_l_share = copy.deepcopy(bs_l)
    for idx, row in bs_l.iterrows():
        bs_l_share.loc[idx, row[row > 0].index] = (per_cash.at[idx] / df['還原收盤價'].fillna(
            method='ffill').loc[idx, row[row > 0].index] / 1000).replace([np.inf, -np.inf, np.nan], 0).astype(int)
    bs_l_daily = pd.DataFrame(index=stk_idx, columns=stk_num)
    bs_l_daily.loc[bs_l_share.index, bs_l_share.columns] = bs_l_share
    bs_l_daily.fillna(method='ffill', inplace=True)
    bs_l_daily = bs_l_daily.shift(1)
    diff = bs_l_daily.diff()
    cost = ((diff[diff > 0].shift(1).fillna(0) *
            df['還原收盤價']*0.001425)*1000).fillna(0)
    cost -= ((diff[diff < 0].fillna(0)*df['還原收盤價']*0.004425)*1000).fillna(0)
    pf_l_daily = bs_l_daily * \
        df['還原收盤價'].fillna(method='ffill').diff().shift(-1)*1000
    pf_l_daily_minus_cost = pf_l_daily - cost

    if ls != 'onlylong':
        bs_s_share = copy.deepcopy(bs_s)
        for idx, row in bs_s.iterrows():
            bs_s_share.loc[idx, row[row < 0].index] = (per_cash.at[idx] / df['還原收盤價'].fillna(
                method='ffill').loc[idx, row[row < 0].index] / 1000).replace([np.inf, -np.inf, np.nan], 0).astype(int)
        bs_s_daily = pd.DataFrame(index=stk_idx, columns=stk_num)
        bs_s_daily.loc[bs_s_share.index, bs_s_share.columns] = -1 * bs_s_share
        bs_s_daily.fillna(method='ffill', inplace=True)
        bs_s_daily = bs_s_daily.shift(1)
        diff = bs_s_daily.diff()
        cost = ((diff[diff > 0].shift(1).fillna(0) *
                df['還原收盤價']*0.001425)*1000).fillna(0)
        cost -= ((diff[diff < 0].fillna(0)*df['還原收盤價']*0.004425)*1000).fillna(0)
        pf_s_daily = bs_s_daily * \
            df['還原收盤價'].fillna(method='ffill').diff().shift(-1)*1000
        pf_s_daily_minus_cost = pf_s_daily - cost



    if ls == 'onlylong':
        sortino = ((250 * pf_l_daily_minus_cost.sum(axis=1).mean()) / (pf_l_daily_minus_cost.sum(axis=1)
                [pf_l_daily_minus_cost.sum(axis=1) < 0].std() * np.sqrt(250))).mean()
    else:
        pf_minus_cost = pf_l_daily_minus_cost + pf_s_daily_minus_cost
        sortino = ((250 * pf_minus_cost.sum(axis=1).mean()) / (pf_minus_cost.sum(axis=1)
                [pf_minus_cost.sum(axis=1) < 0].std() * np.sqrt(250))).mean()
    if pplot:
        if ls == 'onlylong':
            pf_l_daily_minus_cost.sum(axis=1).cumsum().plot(title=f'Sortino: {sortino:.2f}', figsize=(16, 9))
        else:
            pf_minus_cost.sum(axis=1).cumsum().plot(title=f'Sortino: {sortino:.2f}', figsize=(16, 9))
        plt.show()
    if returnbs:
        if ls != 'onlylong':
            return bs_l_daily, bs_s_daily, sortino
        else:
            return bs_l_daily, sortino
    else:
        return sortino
    



In [13]:
# , '資產總計_Q', '負債總計_Q', '現金股利殖利率_Q'
if __name__ == '__main__':
    init()
    data_list = ['本益比(近四季)', '股價淨值比', '成交金額(千)', '還原收盤價',
                 'EPS_Q', '營業收入淨額_Q', '殖利率', '總市值(億)']
    start = datetime.datetime(2004, 1, 1)
    end = datetime.datetime.today()
    data2_list = ['TWA00', '指數彙編', '指數名稱轉換']
    dic = feed.get_from_mongo(elements=data2_list, db='Index')

    df_params = {
        'EPS_Q': '.rolling(4).sum().pct_change().replace([np.inf, -np.inf], 0)',
        '營業收入淨額_Q': '.rolling(4).sum().pct_change().replace([np.inf, -np.inf], 0)',
        '本益比(近四季)': '.mask(data>100, 100)',
        '殖利率': '.mask(data>100, 100)'
    }
    df = feed.get_from_mongo(
        elements=data_list,
        db='Fields',
        symbols=[],
        start=start,
        end=end,
        concat=True,
        ffill=['EPS_Q', '營業收入淨額_Q'],
        zscore=[],
        **df_params)
    # 抓完資料後有些資料會因為fill重複(推估來自於Q轉D)，因此要先調整到正確日期，這邊我選還原收盤價來當作正確日期標準
    df = df.loc[df['還原收盤價'].drop_duplicates().index]

    stk_num = df['還原收盤價'].columns
    stk_len = len(stk_num)
    stk_idx = df['還原收盤價'].index

    # 增加額外Factors
    df = add_factors(name='MA60_diff',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="(df['還原收盤價'] - df['還原收盤價'].rolling(60).mean()) / df['還原收盤價'].rolling(60).std()")
    df = add_factors(name='MA120_diff',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="(df['還原收盤價'] - df['還原收盤價'].rolling(120).mean()) / df['還原收盤價'].rolling(120).std()")
    df = add_factors(name='5MOM',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="(df['還原收盤價'] - df['還原收盤價'].shift(5)) / df['還原收盤價'].shift(5)")
    df = add_factors(name='30MOM',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="(df['還原收盤價'] - df['還原收盤價'].shift(5)) / df['還原收盤價'].shift(5)")
    df = add_factors(name='5std',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="df['還原收盤價'].rolling(5).std()")
    df = add_factors(name='30std',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="df['還原收盤價'].rolling(30).std()")
    df = add_factors(name='bias',
                     df=df, stk_len=stk_len, stk_num=stk_num,
                     func="pd.DataFrame(np.random.randint(-5,5, size=(len(df.index), stk_len)), \
                     index=df['還原收盤價'].index, columns=df['還原收盤價'].columns)")
    data_list.extend(['5MOM', '30MOM', 'bias', '5std', '30std'])

    reverse_list = ['本益比(近四季)', '股價淨值比', '5std', '30std']

    # opt_params = {
    #     '本益比(近四季)': 1,
    #     '股價淨值比': 1,
    #     '成交金額(千)': 1,
    #     '還原收盤價': 1,
    #     'EPS_Q': 1,
    #     '營業收入淨額_Q': 1,
    #     '殖利率': 1,
    #     '總市值(億)': 1,
    #     'MA60_diff': 1,
    #     'MA120_diff': 1,
    #     'bias': 0
    # }
    opt_params = {k: 1 for k in df.columns.get_level_values(0).unique()}
    
    df_value = calculate_value(
        df=df, dic=dic, data_list=data_list, 
        reverse_list=reverse_list, stk_len=stk_len, stk_num=stk_num)

    scores = calculate_scores(
        df_value=df_value, stk_idx=stk_idx, stk_num=stk_num, **opt_params)
        



Data TWA00 has shape (6056, 34)
Data 指數彙編 has shape (2211, 8)
Data 指數名稱轉換 has shape (1, 22)
Data 本益比(近四季) has shape (4678, 2211)
Data 股價淨值比 has shape (4678, 2211)
Data 成交金額(千) has shape (4678, 2211)
Data 還原收盤價 has shape (4678, 2211)
Data EPS_Q has shape (76, 2211)
Data 營業收入淨額_Q has shape (76, 2211)
Data 殖利率 has shape (4678, 2211)
Data 總市值(億) has shape (4678, 2211)


In [14]:
import pymongo as pm
import pandas as pd
from typing import Union
import datetime
import numpy as np
import vectorbt as vbt
import talib
from numba import njit
import gc

def get_from_mongo(elements: list, symbols: list=[], db:str='Fields',
    start: Union[datetime.datetime, None]=None, 
    end: Union[datetime.datetime, None]=None,
    concat: bool=False,
    ffill: list=[],
    zscore: list=[],
    pprint: bool=True,
    **kwards) -> Union[pd.DataFrame, dict]:
    """
    elements: Element, 表示要在Mongo中抓哪些資料, 須用list包起來
    symbols: 商品清單
    start: 開始時間, 若沒定義則會抓取沒有日期的資料
    end: 結束時間
    concat: 打開以後就會return一個大表，擁有multiindex的columns，level0是表單名稱(開盤價、收盤價等)
    """
    client = pm.MongoClient()
    if concat:
        df = pd.DataFrame()
    else:
        df = dict()
    symbols = {s: 1 for s in symbols}
    symbols['_id'] = 0
    if start == None:
        date = {}
    else:
        date = {'日期': {'$gte': start, '$lte': end}}
        if len(symbols) != 1:
            symbols['日期'] = 1

    for e in elements:
        data = pd.DataFrame(client[db][e].find(date, symbols))
        if '日期' in data.columns:
            data.set_index('日期', inplace=True)
        data = data.reindex(sorted(data.columns), axis=1)
        if pprint:
            print(f'Data {e} has shape {data.shape}')
        data.columns = pd.MultiIndex.from_tuples(tuple(zip([e]*len(data.columns), data.columns)))
        # 資料預先處理就放在**kwards裡
        if e in kwards: # 'Q': .rolling(4).sum()
            string = 'data'+kwards[e]
            data = eval(string)
        if e in zscore:
            data = (data.subtract(data.mean(axis=1), axis=0)).divide(data.std(ddof=0, axis=1), axis=0)
            data = data.mask(data>5, 5)
            data = data.mask(data<-5, -5)
        if concat:
            df = pd.concat([df, data], axis=1, join='outer')
        else:
            df[e] = data
    if concat:
        df.sort_index(inplace=True)
    for f in ffill:
        df.loc[:, f] = df.ffill()
    return df

In [2]:
client = pm.MongoClient()

data = pd.DataFrame(client['Fields']['還原收盤價'].find(
    {'日期': {'$gte': datetime.datetime(2006, 1, 1), 
    '$lte': datetime.datetime.today()}}, {'_id': 0})).set_index('日期')
pb = pd.DataFrame(client['Fields']['股價淨值比'].find(
    {'日期': {'$gte': datetime.datetime(2006, 1, 1), 
    '$lte': datetime.datetime.today()}}, {'_id': 0})).set_index('日期')


In [3]:
data.dropna(axis=1, how='all', inplace=True)
data = data.fillna(method='ffill').fillna(method='bfill').replace(to_replace=0, method='ffill')
data.columns.name = "symbol"
pb.dropna(axis=1, how='all', inplace=True)
pb = pb.fillna(method='ffill').fillna(method='bfill').replace(to_replace=0, method='ffill')
pb.columns.name = "symbol"
pb = pb.align(data, join="right")[0]

In [4]:
data, indexes = data.vbt.range_split(n=25, range_len=1*250)
pb, indexespb = pb.vbt.range_split(n=25, range_len=1*250)

In [66]:
score_rank = np.zeros(scores.shape)
for e in enumerate(np.argsort(scores.to_numpy(), axis=1)[:, -1:]):
    score_rank[e] = 1


In [115]:
scores.rank(axis=1, ascending=False).iloc[10, 289]

1.0

In [117]:
score = scores.rank(axis=1, ascending=False)
score.columns.name = "symbol"
score, score_idx = score.vbt.range_split(n=25, range_len=250)

In [118]:
data = df['還原收盤價'].fillna(method='ffill').fillna(method='bfill').replace(to_replace=0, method='ffill')
data.columns.name = "symbol"
data, data_idx = data.vbt.range_split(n=25, range_len=250)




In [119]:

@njit
def optimize_rsi(close, scores, entry, exit):
    return scores <= entry, scores > exit

rsi_ind = vbt.IndicatorFactory(
        class_name="ScoresRanking",
        short_name="Sc",
        input_names=["close", "scores"],
        param_names=["entry", "exit"],
        output_names=["entries", "exits"]
        ).from_apply_func(
            optimize_rsi,
            entry=50,
            exit=100)

step_size = 1
entries = np.arange(1, 51, step=step_size, dtype=int)
exits = np.arange(52, 101, step=step_size, dtype=int)

master_pf = list()
for entry in entries:
    for exit in exits:
        for i in range(25):
            gc.collect()
            score_res = rsi_ind.run(
                    data[i],
                    score[i],
                    entry=entry,
                    exit=exit,
                    param_product=True)

            score_entries = score_res.entries
            score_exits = score_res.exits

            score_exits.iloc[-1, :] = True

            rsi_pf = vbt.Portfolio.from_signals(data[i], score_entries, score_exits, freq="1D", fees=0.001425)
            master_pf.append(rsi_pf.total_return())

            # fig = rsi_pf.total_return().vbt.volume(
            #         x_level="rsi_exit",
            #         y_level="rsi_entry",
            #         z_level="rsi_window",
            #         slider_level="split_idx"
            # )
            # fig.show()

In [9]:
master = pd.concat(master_pf).groupby(["Sc_entry", "Sc_exit"]).mean()
fig = master.vbt.heatmap(
        x_level="Sc_entry",
        y_level="Sc_exit"
)

fig.show()

# 挑越多股票越好

In [120]:
master = pd.concat(master_pf).groupby(["Sc_entry", "Sc_exit"]).mean()
fig = master.vbt.heatmap(
        x_level="Sc_entry",
        y_level="Sc_exit"
)

fig.show()

In [5]:

RSI = vbt.IndicatorFactory.from_talib('RSI')


@njit
def produce_signal(rsi, entry, exit):
    trend = np.where((rsi > exit), -1, 0)
    trend = np.where((rsi < entry) , 1, trend)
    return trend

def custom_indicator(close, rsi_window=14, entry = 30, exit = 70):
    rsi =  RSI.run(close, rsi_window).real.to_numpy()
    #rsi = vbt.RSI.run(close, window=rsi_window).rsi

    return produce_signal(rsi, entry, exit)

ind = vbt.IndicatorFactory(
    class_name = "Combination",
    short_name = "Comb",
    input_names = ["close"],
    param_names = ["rsi_window","entry", "exit"],
    output_names = ["value"],
).from_apply_func(
    custom_indicator,
    rsi_window = 14,
    entry=30,
    exit=70,
)


res = ind.run(data, 
        rsi_window=np.arange(10, 33, step=22, dtype=int), 
        entry=np.arange(20, 40, step=5, dtype=int),
        exit=np.arange(20, 40, step=5, dtype=int),
        param_product=True)

#print(res.value)

entries = res.value == 1.0
exits = res.value == -1.0

pf = vbt.Portfolio.from_signals(data, entries, exits)
print(pf.stats())
print(pf.total_return())

c:\Users\ChastLai\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\ChastLai\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\ChastLai\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\ChastLai\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2006-01-02 00:00:00
End                           2010-12-31 00:00:00
Period                                       1246
Start Value                                 100.0
End Value                               91.736734
Total Return [%]                        -8.263266
Benchmark Return [%]                    96.276706
Max Gross Exposure [%]                  79.287739
Total Fees Paid                               0.0
Max Drawdown [%]                        31.701501
Max Drawdown Duration                  631.760939
Total Trades                            11.307561
Total Closed Trades                     11.248516
Total Open Trades                        0.059045
Open Trade PnL                          -0.762628
Win Rate [%]                            60.414129
Best Trade [%]                           6.442951
Worst Trade [%]                        -14.760875
Avg Winning Trade [%]                    4.206344
Avg Losing Trade [%]                    -8.228564


C:\Users\ChastLai\AppData\Local\Temp\ipykernel_22664\3441461525.py:41: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x000001E5456141F0>. Pass column to select a single column/group.
  print(pf.stats())


In [9]:
returns = pf.total_return()
print(returns.max())
print(returns.idxmax())

38.464342538561716
(10, 20, 30, 20, '3021')


In [10]:
#returns = returns[ returns.index.isin(["2330"], level="symbol")]
returns = returns.groupby(level=["Comb_rsi_window", "Comb_ma_window", "Comb_entry"]).mean()
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400
fig = returns.vbt.volume(
    x_level = "Comb_rsi_window",
    y_level = "Comb_ma_window",
    z_level = "Comb_entry",
    #slider_level = "symbol",
)
fig.show()

Comb_rsi_window  Comb_ma_window
10               20               -0.151016
                 30               -0.165269
                 40               -0.156064
                 50               -0.162152
                 60               -0.189684
                 70               -0.195346
                 80               -0.203998
                 90               -0.202770
                 100              -0.199646
                 110              -0.191384
                 120              -0.193381
                 130              -0.196564
                 140              -0.207516
                 150              -0.214831
                 160              -0.214053
                 170              -0.211327
                 180              -0.224752
                 190              -0.223956
20               20               -0.051507
                 30               -0.053068
                 40               -0.056568
                 50               -0.068097
